# Qubitization Walk Operator

Bloqs for constructing quantum walks from Select and Prepare operators.

The spectrum of a quantum Hamiltonian can be encoded in the spectrum of a quantum "walk"
operator. The Prepare and Select subroutines are carefully designed so that the Hamiltonian
$H$ is encoded as a projection of Select onto the state prepared by Prepare:

$$
\mathrm{PREPARE}|0\rangle = |\mathcal{L}\rangle \\
(\langle \mathcal{L} | \otimes \mathbb{1}) \mathrm{SELECT} (|\mathcal{L} \rangle \otimes \mathbb{1}) = H / \lambda
$$.

We first document the SelectOracle and PrepareOracle abstract base bloqs, and then show
how they can be combined in `QubitizationWalkOperator`.

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `SelectOracle`
Abstract base class that defines the interface for a SELECT Oracle.

The action of a SELECT oracle on a selection register $|l\rangle$ and target register
$|\Psi\rangle$ can be defined as:

$$
    \mathrm{SELECT} = \sum_{l}|l \rangle \langle l| \otimes U_l
$$

In other words, the `SELECT` oracle applies $l$'th unitary $U_{l}$ on the target register
$|\Psi\rangle$ when the selection register stores integer $l$.

$$
    \mathrm{SELECT}|l\rangle |\Psi\rangle = |l\rangle U_{l}|\Psi\rangle
$$

In [ ]:
from qualtran.bloqs.select_and_prepare import SelectOracle

## `PrepareOracle`
Abstract base class that defines the API for a PREPARE Oracle.

Given a set of coefficients $\{c_0, c_1, ..., c_{N - 1}\}$, the PREPARE oracle is used to encode
the coefficients as amplitudes of a state $|\Psi\rangle = \sum_{i=0}^{N-1} \sqrt{\frac{c_{i}}{\lambda}} |i\rangle$
where $\lambda = \sum_i |c_i|$, using a selection register $|i\rangle$. In order to prepare such
a state, the PREPARE circuit is also allowed to use a junk register that is entangled with
selection register.

Thus, the action of a PREPARE circuit on an input state $|0\rangle$ can be defined as:

$$
    \mathrm{PREPARE} |0\rangle = \sum_{i=0}^{N-1} \sqrt{ \frac{c_{i}}{\lambda} } |i\rangle |junk_{i}\rangle
$$

In [ ]:
from qualtran.bloqs.select_and_prepare import PrepareOracle

## `QubitizationWalkOperator`
Constructs a Szegedy Quantum Walk operator using LCU oracles SELECT and PREPARE.

For a Hamiltonian $H = \sum_l w_l H_l$ (s.t. $w_l > 0$ and $H_l$ are unitaries),
Constructs a Szegedy quantum walk operator $W = R_{L} . SELECT$, which is a product of
two reflections $R_{L} = (2|L><L| - I)$ and $SELECT=\sum_{l}|l><l|H_{l}$.

The action of $W$ partitions the Hilbert space into a direct sum of two-dimensional irreducible
    vector spaces. For an arbitrary eigenstate $|k>$ of $H$ with eigenvalue $E_k$, $|\ell>|k>$ and
an orthogonal state $\phi_{k}$ span the irreducible two-dimensional space that $|\ell>|k>$ is
in under the action of $W$. In this space, $W$ implements a Pauli-Y rotation by an angle of
$-2arccos(E_{k} / \lambda)$ s.t. $W = e^{i arccos(E_k / \lambda) Y}$,
where $\lambda = \sum_l w_l$.

Thus, the walk operator $W$ encodes the spectrum of $H$ as a function of eigenphases of $W$
s.t. $spectrum(H) = \lambda cos(\arg(\mathrm{spectrum}(W)))$ where $\arg(e^{i\phi}) = \phi$.

#### Parameters
 - `select`: The SELECT lcu gate implementing $\mathrm{SELECT}=\sum_{l}|l\rangle\langle l|H_{l}$.
 - `prepare`: Then PREPARE lcu gate implementing $\mathrm{PREPARE}|00...00\rangle = \sum_{l=0}^{L - 1}\sqrt{\frac{w_{l}}{\lambda}} |l\rangle = |\ell\rangle$
 - `control_val`: If 0/1, a controlled version of the walk operator is constructed. Defaults to None, in which case the resulting walk operator is not controlled. 

#### References
 - [Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity](https://arxiv.org/abs/1805.03662). Babbush et. al. (2018). Figure 1.


In [ ]:
from qualtran.bloqs.qubitization_walk_operator import QubitizationWalkOperator

### Example Instances

In [ ]:
from qualtran.bloqs.qubitization_walk_operator_test import get_walk_operator_for_1d_ising_model

walk_op = get_walk_operator_for_1d_ising_model(4, 2e-1)

In [ ]:
from openfermion.resource_estimates.utils import QI

from qualtran.bloqs.chemistry.thc.walk_operator import get_walk_operator_for_thc_ham

# Li et al parameters from openfermion.resource_estimates.thc.compute_cost_thc_test
num_spinorb = 152
num_bits_state_prep = 10
num_bits_rot = 20
thc_dim = 450
num_spat = num_spinorb // 2
tpq = np.random.normal(size=(num_spat, num_spat))
tpq = 0.5 * (tpq + tpq) / 2
zeta = np.random.normal(size=(thc_dim, thc_dim))
zeta = 0.5 * (zeta + zeta) / 2
qroam_blocking_factor = np.power(2, QI(thc_dim + num_spat)[0])
thc_walk_op = get_walk_operator_for_thc_ham(
    tpq,
    zeta,
    num_bits_state_prep=num_bits_state_prep,
    num_bits_theta=num_bits_rot,
    kr1=qroam_blocking_factor,
    kr2=qroam_blocking_factor,
)

In [ ]:
from qualtran.bloqs.chemistry.sparse.prepare_test import build_random_test_integrals
from qualtran.bloqs.chemistry.sparse.walk_operator import get_walk_operator_for_sparse_chem_ham

num_spin_orb = 8
num_bits_rot_aa = 8
num_bits_state_prep = 12
tpq, eris = build_random_test_integrals(num_spin_orb // 2)
walk_op_chem_sparse = get_walk_operator_for_sparse_chem_ham(
    tpq, eris, num_bits_rot_aa=num_bits_rot_aa, num_bits_state_prep=num_bits_state_prep
)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([walk_op, thc_walk_op, walk_op_chem_sparse],
           ['`walk_op`', '`thc_walk_op`', '`walk_op_chem_sparse`'])

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
walk_op_g, walk_op_sigma = walk_op.call_graph(max_depth=1, generalizer=ignore_split_join)
show_call_graph(walk_op_g)
show_counts_sigma(walk_op_sigma)